# FLIP(01):  Advanced Data Science
**(Module 03: Natural Language Processing)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, but NOT allowed to change or distribute this package.

Prepared by and for 
**Student Members** |
2006-2021 [TULIP Lab](http://www.tulip.org.au)

---


# Session 09 - Analyzing the Meaning of Sentences

### Natural Language Understanding

#### Querying a Database

In this section,we will show that solving the task in a restricted domain is pretty straightforward. But we will also see that to address the problem in a more general way, we have to open up a whole new box of ideas and techniques, involving the representation of meaning.

In [ ]:
import nltk

In [ ]:
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

In [ ]:
from nltk import load_parser

In [ ]:
cp = load_parser('grammars/book_grammars/sql0.fcfg')

In [ ]:
query = 'What cities are located in China'

In [ ]:
trees = list(cp.parse(query.split()))

In [ ]:
answer = trees[0].label()['SEM']

In [ ]:
answer = [s for s in answer if s]

In [ ]:
q = ' '.join(answer)

In [ ]:
print(q)

Finally, we execute the query over the database city.db and retrieve some results.



In [ ]:
from nltk.sem import chat80

In [ ]:
rows = chat80.sql_query('corpora/city_database/city.db', q)

In [ ]:
for r in rows: print(r[0], end=" ")

# Propositional Logic

A logical language is designed to make reasoning formally explicit. As a result, it can capture aspects of natural language which determine whether a set of sentences is consistent. As part of this approach, we need to develop logical representations of a sentence φ that formally capture the truth-conditions of φ.

In [ ]:
nltk.boolean_ops()

NLTKs Expression object can process logical expressions into various subclasses of Expression:



In [ ]:
read_expr = nltk.sem.Expression.fromstring

In [ ]:
read_expr('-(P & Q)')

In [ ]:
read_expr('P & Q')

In [ ]:
read_expr('P | (R -> Q)')

In [ ]:
read_expr('P <-> -- P')

Arguments can be tested for "syntactic validity" by using a proof system. We will say a little bit more about this later. Logical proofs can be carried out with NLTK's inference module, for example via an interface to the third-party theorem prover Prover9. The inputs to the inference mechanism first have to be converted into logical expressions.

In [ ]:
lp = nltk.sem.Expression.fromstring

In [ ]:
SnF = read_expr('SnF')

In [ ]:
NotFnS = read_expr('-FnS')

In [ ]:
R = read_expr('SnF -> -FnS')

In [ ]:
prover = nltk.Prover9()

In [ ]:
prover.prove(NotFnS, [SnF, R])

Recall that we interpret sentences of a logical language relative to a model, which is a very simplified version of the world. A model for propositional logic needs to assign the values True or False to every possible formula. We do this inductively: first, every propositional symbol is assigned a value, and then we compute the value of complex formulas by consulting the meanings of the boolean operators and applying them to the values of the formula's components. A Valuation is a mapping from basic expressions of the logic to their values. Here's an example:

In [ ]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])

In [ ]:
val['P']

In [ ]:
dom = set()

In [ ]:
g = nltk.Assignment(dom)

In [ ]:
m = nltk.Model(dom, val)

In [ ]:
print(m.evaluate('(P & Q)', g))

In [ ]:
print(m.evaluate('-(P & Q)', g))

In [ ]:
print(m.evaluate('(P & R)', g))

In [ ]:
print(m.evaluate('(P | R)', g))

# First-Order Logic

In the remainder of this chapter, we will represent the meaning of natural language expressions by translating them into first-order logic. Not all of natural language semantics can be expressed in first-order logic. But it is a good choice for computational semantics because it is expressive enough to represent many aspects of semantics, and on the other hand, there are excellent systems available off the shelf for carrying out automated inference in first-order logic.

## Syntax

First-order logic keeps all the Boolean operators of propositional logic, but it adds some important new mechanisms.

It is often helpful to inspect the syntactic structure of expressions of first-order logic, and the usual way of doing this is to assign types to expressions. Following the tradition of Montague grammar, we will use two basic types: e is the type of entities, while t is the type of formulas, i.e., expressions which have truth values. Given these two basic types, we can form complex types for function expressions. That is, given any types σ and τ, 〈σ, τ〉 is a complex type corresponding to functions from 'σ things' to 'τ things'. For example, 〈e, t〉 is the type of expressions from entities to truth values, namely unary predicates. The logical expression can be processed with type checking.

In [ ]:
read_expr = nltk.sem.Expression.fromstring

In [ ]:
expr = read_expr('walk(angus)', type_check=True)

In [ ]:
expr.argument

In [ ]:
expr.argument.type

In [ ]:
expr.function

In [ ]:
expr.function.type

Why do we see <e,?> at the end of this example? Although the type-checker will try to infer as many types as possible, in this case it has not managed to fully specify the type of walk, since its result type is unknown. Although we are intending walk to receive type <e, t>, as far as the type-checker knows, in this context it could be of some other type such as <e, e> or <e, <e, t>. To help the type-checker, we need to specify a signature, implemented as a dictionary that explicitly associates types with non-logical constants:

In [ ]:
sig = {'walk': '<e, t>'}

In [ ]:
expr = read_expr('walk(angus)', signature=sig)

In [ ]:
expr.function.type

In [ ]:
read_expr = nltk.sem.Expression.fromstring

In [ ]:
read_expr('dog(cyril)').free()

In [ ]:
read_expr('dog(x)').free()

In [ ]:
read_expr('own(angus, cyril)').free()

In [ ]:
read_expr('exists x.dog(x)').free()

In [ ]:
read_expr('((some x. walk(x)) -> sing(x))').free()

In [ ]:
read_expr('exists x.own(y, x)').free()

## First-Order Theorem Proving

The general case in theorem proving is to determine whether a formula that we want to prove (a proof goal) can be derived by a finite sequence of inference steps from a list of assumed formulas.

In [ ]:
NotFnS = read_expr('-north_of(f, s)')

In [ ]:
SnF = read_expr('north_of(s, f)')

In [ ]:
R = read_expr('all x. all y. (north_of(x, y) -> -north_of(y, x))')

In [ ]:
prover = nltk.Prover9()

In [ ]:
prover.prove(NotFnS, [SnF, R])

In [ ]:
FnS = read_expr('north_of(f, s)')

In [ ]:
prover.prove(FnS, [SnF, R])

## Truth in Model

Relations are represented semantically in NLTK in the standard set-theoretic way: as sets of tuples. For example, let’s suppose we have a domain of discourse consisting of the individuals Bertie, Olive, and Cyril, where Bertie is a boy, Olive is a girl, and Cyril is a dog.

In [ ]:
dom = {'b', 'o', 'c'}

In [ ]:
v = """
    bertie => b
    olive => o
    cyril => c
    boy => {b}
    girl => {o}
    dog => {c}
    walk => {o, c}
    see => {(b, o), (c, b), (o, c)}
    """

In [ ]:
val = nltk.Valuation.fromstring(v)

In [ ]:
print(val)

You may have noticed that our unary predicates (i.e, boy, girl, dog) also come out as sets of singleton tuples, rather than just sets of individuals. This is a convenience which allows us to have a uniform treatment of relations of any arity. A predication of the form P(τ1, ... τn), where P is of arity n, comes out true just in case the tuple of values corresponding to (τ1, ... τn) belongs to the set of tuples in the value of P.

In [ ]:
('o', 'c') in val['see']

In [ ]:
('b',) in val['boy']

## Individual Variables and Assignments

In our models, the counterpart of a context of use is a variable assignment. This is a mapping from individual variables to entities in the domain. Assignments are created using the Assignment constructor, which also takes the model’s domain of discourse as a parameter.

In [ ]:
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])

In [ ]:
print(g)

In [ ]:
m = nltk.Model(dom, val)

In [ ]:
m.evaluate('see(olive, y)', g)

In [ ]:
g['y']

In [ ]:
m.evaluate('see(y, x)', g)

In [ ]:
g.purge()

In [ ]:
m.evaluate('see(olive, y)', g)

In [ ]:
m.evaluate('see(bertie, olive) & boy(bertie) & -walk(bertie)', g)

## Quantification

One of the crucial insights of modern logic is that the notion of variable satisfaction can be used to provide an interpretation for quantified formulas.

In [ ]:
m.evaluate('exists x.(girl(x) & walk(x))', g)

In [ ]:
m.evaluate('girl(x) & walk(x)', g.add('x', 'o'))

In [ ]:
fmla1 = read_expr('girl(x) | boy(x)')

In [ ]:
m.satisfiers(fmla1, 'x', g)

In [ ]:
fmla2 = read_expr('girl(x) -> walk(x)')

In [ ]:
m.satisfiers(fmla2, 'x', g)

In [ ]:
fmla3 = read_expr('walk(x) -> girl(x)')

In [ ]:
m.satisfiers(fmla3, 'x', g)

In [ ]:
m.evaluate('all x.(girl(x) -> walk(x))', g)

## Quantifier Scope Ambiguity

In [ ]:
v2 = """
    bruce => b
    cyril => c
    elspeth => e
    julia => j
    matthew => m
    person => {b, e, j, m}
    admire => {(j, b), (b, b), (m, e), (e, m), (c, a)}
    """

In [ ]:
val2 = nltk.Valuation.fromstring(v2)

In [ ]:
dom2 = val2.domain

In [ ]:
m2 = nltk.Model(dom2, val2)

In [ ]:
g2 = nltk.Assignment(dom2)

In [ ]:
fmla4 = read_expr('(person(x) -> exists y.(person(y) & admire(x, y)))')

In [ ]:
m2.satisfiers(fmla4, 'x', g2)

This shows that fmla4 holds of every individual in the domain. By contrast, consider the formula fmla5 below; this has no satisfiers for the variable y.

In [ ]:
fmla5 = read_expr('(person(y) & all x.(person(x) -> admire(x, y)))')

In [ ]:
m2.satisfiers(fmla5, 'y', g2)

In [ ]:
fmla6 = read_expr('(person(y) & all x.((x = bruce | x = julia) -> admire(x, y)))')

In [ ]:
m2.satisfiers(fmla6, 'y', g2)

## Model Building

We have been assuming that we already had a model, and wanted to check the truth of a sentence in the model. By contrast, model building tries to create a new model,given some set of sentences. If it succeeds, then we know that the set is consistent, since we have an existence proof of the model.

In [ ]:
a3 = read_expr('exists x.(man(x) & walks(x))')

In [ ]:
c1 = read_expr('mortal(socrates)')

In [ ]:
c2 = read_expr('-mortal(socrates)')

In [ ]:
mb = nltk.Mace(5)

In [ ]:
print(mb.build_model(None, [a3, c1]))

In [ ]:
print(mb.build_model(None, [a3, c2]))

In [ ]:
print(mb.build_model(None, [c1, c2]))

In [ ]:
a4 = read_expr('exists y. (woman(y) & all x. (man(x) -> love(x,y)))')

In [ ]:
a5 = read_expr('man(adam)')

In [ ]:
a6 = read_expr('woman(eve)')

In [ ]:
g = read_expr('love(adam,eve)')

In [ ]:
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6])

In [ ]:
mc.build_model()

In [ ]:
print(mc.valuation)

In [ ]:
a7 = read_expr('all x. (man(x) -> -woman(x))')

In [ ]:
g = read_expr('love(adam,eve)')

In [ ]:
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6, a7])

In [ ]:
mc.build_model()

In [ ]:
print(mc.valuation)

# The Semantics of English Sentences

## The λ-Calculus

Remember that \ is a special character in Python strings. We must either escape it (with another \), or else use “raw strings”  as shown here:

In [ ]:
read_expr = nltk.sem.Expression.fromstring

In [ ]:
expr = read_expr(r'\x.(walk(x) & chew_gum(x))')

In [ ]:
expr

In [ ]:
expr

In [ ]:
print(read_expr(r'\x.(walk(x) & chew_gum(y))'))

In [ ]:
expr = read_expr(r'\x.(walk(x) & chew_gum(x))(gerald)')

In [ ]:
print(expr)

In [ ]:
print(expr.simplify())

In [ ]:
print(read_expr(r'\x.\y.(dog(x) & own(y, x))(cyril)').simplify())

In [ ]:
print(read_expr(r'\x y.(dog(x) & own(y, x))(cyril, angus)').simplify())

In [ ]:
expr1 = read_expr('exists x.P(x)')

In [ ]:
print(expr1)

In [ ]:
expr2 = expr1.alpha_convert(nltk.sem.Variable('z'))

In [ ]:
print(expr2)

In [ ]:
expr1 == expr2

In [ ]:
expr3 = read_expr('\P.(exists x.P(x))(\y.see(y, x))')

In [ ]:
print(expr3)

In [ ]:
print(expr3.simplify())

## Quantified NPs

At the start of this section, we briefly described how to build a semantic representation for Cyril barks. You would be forgiven for thinking this was all too easy—surely there is a bit more to building compositional semantics.

In [ ]:
# lp = nltk.LogicParser()

In [ ]:
# tvp = lp.parse(r'\X x.X(\y.chase(x,y))')

In [ ]:
# np = lp.parse(r'(\P.exists x.(dog(x) & P(x)))')

In [ ]:
# vp = nltk.ApplicationExpression(tvp, np)

In [ ]:
# print vp

In [ ]:
# print vp.simplify()

In [ ]:
# from nltk import load_parser

In [ ]:
# parser = load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)

In [ ]:
# sentence = 'Angus gives a bone to every dog'

In [ ]:
# tokens = sentence.split()

In [ ]:
# trees = parser.nbest_parse(tokens)

In [ ]:
# for tree in trees:
    # print tree.node['SEM']

In [ ]:
# v = """
#     bertie => b
#     olive => o
#     cyril => c
#     boy => {b}
#     girl => {o}
#     dog => {c}
#     walk => {o, c}
#     see => {(b, o), (c, b), (o, c)}
#     """

In [ ]:
# val = nltk.parse_valuation(v)

In [ ]:
# g = nltk.Assignment(val.domain)

In [ ]:
# m = nltk.Model(val.domain, val)

In [ ]:
# sent = 'Cyril sees every boy'

In [ ]:
# grammar_file = 'grammars/book_grammars/simple-sem.fcfg'

In [ ]:
# results = nltk.batch_evaluate([sent], grammar_file, m, g)[0]

In [ ]:
# for (syntree, semrel, value) in results:
    # print semrep
    # print value

## Transitive Verbs

Our next challenge is to deal with sentences containing transitive verbs

In [ ]:
read_expr = nltk.sem.Expression.fromstring

In [ ]:
tvp = read_expr(r'\X x.X(\y.chase(x,y))')

In [ ]:
np = read_expr(r'(\P.exists x.(dog(x) & P(x)))')

In [ ]:
vp = nltk.sem.ApplicationExpression(tvp, np)

In [ ]:
print(vp)

In [ ]:
print(vp.simplify())

The grammar simple-sem.fcfg contains a small set of rules for parsing and translating simple examples of the kind that we have been looking at. Here's a slightly more complicated example.

In [ ]:
from nltk import load_parser

In [ ]:
parser = load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)

In [ ]:
sentence = 'Angus gives a bone to every dog'

In [ ]:
tokens = sentence.split()

In [ ]:
for tree in parser.parse(tokens):
    print(tree.label()['SEM'])

NLTK provides some utilities to make it easier to derive and inspect semantic interpretations. The function interpret_sents() is intended for interpretation of a list of input sentences. It builds a dictionary d where for each sentence sent in the input, d[sent] is a list of pairs (synrep, semrep) consisting of trees and semantic representations for sent. The value is a list since sent may be syntactically ambiguous; in the following example, however, there is only one parse tree per sentence in the list.

In [ ]:
sents = ['Irene walks', 'Cyril bites an ankle']

In [ ]:
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'

In [ ]:
for results in nltk.interpret_sents(sents, grammar_file):
    for (synrep, semrep) in results:
        print(synrep)

We have seen now how to convert English sentences into logical forms, and earlier we saw how logical forms could be checked as true or false in a model. Putting these two mappings together, we can check the truth value of English sentences in a given model. Let's take model m as defined above. The utility evaluate_sents() resembles interpret_sents() except that we need to pass a model and a variable assignment as parameters. The output is a triple (synrep, semrep, value) where synrep, semrep are as before, and value is a truth value. For simplicity, the following example only processes a single sentence.

In [ ]:
v = """
... bertie => b
... olive => o
... cyril => c
... boy => {b}
... girl => {o}
... dog => {c}
... walk => {o, c}
... see => {(b, o), (c, b), (o, c)}
... """

In [ ]:
val = nltk.Valuation.fromstring(v)

In [ ]:
g = nltk.Assignment(val.domain)

In [ ]:
m = nltk.Model(val.domain, val)

In [ ]:
sent = 'Cyril sees every boy'

In [ ]:
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'

In [ ]:
results = nltk.evaluate_sents([sent], grammar_file, m, g)[0]

In [ ]:
for (syntree, semrep, value) in results:
    print(semrep)
    print(value)

## Quantifier Ambiguity Revisited

One important limitation of the methods described earlier is that they do not deal with scope ambiguity.

In [ ]:
from nltk.sem import cooper_storage as cs

In [ ]:
sentence = 'every girl chases a dog'

In [ ]:
trees = cs.parse_with_bindops(sentence, grammar='grammars/book_grammars/storage.fcfg')

In [ ]:
semrep = trees[0].label()['SEM']

In [ ]:
cs_semrep = cs.CooperStore(semrep)

In [ ]:
print(cs_semrep.core)

In [ ]:
for bo in cs_semrep.store:
    print(bo)

In [ ]:
cs_semrep.s_retrieve(trace=True)

In [ ]:
for reading in cs_semrep.readings:
    print(reading)

# Discourse Semantics

A discourse is a sequence of sentences. Very often, the interpretation of a sentence in a discourse depends on what preceded it. A clear example of this comes from anaphoric pronouns, such as he, she, and it. Given a discourse such as Angus used to have a dog.
But he recently disappeared., you will probably interpret he as referring to Angus’s dog. However, in Angus used to have a dog. He took him for walks in New Town., you are more likely to interpret he as referring to Angus himself.

## Discourse Representation Theory

In [ ]:
read_dexpr = nltk.sem.DrtExpression.fromstring

In [ ]:
drs1 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')

In [ ]:
drs1.draw()

In [ ]:
print(drs1.fol())

In [ ]:
drs2 = read_dexpr('([x], [walk(x)]) + ([y], [run(y)])')

In [ ]:
print(drs2)

In [ ]:
print(drs2.simplify())

In [ ]:
drs3 = read_dexpr('([], [(([x], [dog(x)]) -> ([y],[ankle(y), bite(x, y)]))])')

In [ ]:
print(drs3.fol())

In [ ]:
drs4 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')

In [ ]:
drs5 = read_dexpr('([u, z], [PRO(u), irene(z), bite(u, z)])')

In [ ]:
drs6 = drs4 + drs5

In [ ]:
print(drs6.simplify())

In [ ]:
print(drs6.simplify().resolve_anaphora())

In [ ]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/drt.fcfg', logic_parser=nltk.sem.drt.DrtParser())

In [ ]:
trees = list(parser.parse('Angus owns a dog'.split()))

In [ ]:
print(trees[0].label()['SEM'].simplify())

## Discourse Processing

When we interpret a sentence, we use a rich context for interpretation, determined in part by the preceding context and in part by our background assumptions.

In [ ]:
dt = nltk.DiscourseTester(['A student dances', 'Every student is a person'])

In [ ]:
dt.readings()

When a new sentence is added to the current discourse, setting the parameter consistchk=True causes consistency to be checked by invoking the model checker for each thread, i.e., sequence of admissible readings. In this case, the user has the option of retracting the sentence in question.

In [ ]:
dt.add_sentence('No person dances', consistchk=True)

In [ ]:
dt.retract_sentence('No person dances', verbose=True)

In [ ]:
dt.add_sentence('A person dances', informchk=True)

In [ ]:
from nltk.tag import RegexpTagger
tagger = RegexpTagger(
                    [('^(chases|runs)$', 'VB'),
                    ('^(a)$', 'ex_quant'),
                    ('^(every)$', 'univ_quant'),
                    ('^(dog|boy)$', 'NN'),
                    ('^(He)$', 'PRP')
                    ])

In [ ]:
rc = nltk.DrtGlueReadingCommand(depparser=nltk.MaltParser(tagger=tagger))

In [ ]:
dt = nltk.DiscourseTester(['Every dog chases a boy', 'He runs'], rc)

In [ ]:
dt.readings()

In [ ]:
dt.readings(show_thread_readings=True)

In [ ]:
dt.readings(show_thread_readings=True, filter=True)